In [1]:
from numpy import *

def loadDataSet(filename):
	dataMat = []
	fr = open(filename)
	for line in fr.readlines():
		curLine = line.strip().split('\t')
		fltLine = list(map(float,curLine))#保存为浮点型
		dataMat.append(fltLine)
	return dataMat

#生成叶节点
def regLeaf(dataSet):
	return mean(dataSet[:,-1])

#目标变量的平方误差
def regErr(dataSet):
	return var(dataSet[:,-1]) * shape(dataSet)[0]

def chooseBestSplit(dataSet, leafType = regLeaf, errType = regErr, ops = (1,4)):
	tolS = ops[0]#容许的误差下降值
	tolN = ops[1]#切分的最少样本数
	#统计不同剩余特征值的数目，如果该数目为1，就不在切分而直接返回
	if len(set(dataSet[:,-1].T.tolist()[0])) == 1:
		return None, leafType(dataSet)

	#数据集大小
	m,n = shape(dataSet)
	#数据集误差，用于与新切分误差进行对比，检查新切分能否降低误差
	S = errType(dataSet)
	bestS = inf 
	bestIndex = 0
	bestValue = 0
	for featIndex in range(n-1):
		for splitVal in set(dataSet[:,featIndex].T.tolist()[0]):
			mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
			if(shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
				continue
			newS = errType(mat0) + errType(mat1)
			if newS < bestS:
				bestIndex = featIndex
				bestValue = splitVal
				bestS = newS
	#切分数据集后效果提升不够大，则不进行切分而直接创建叶节点
	if (S - bestS) < tolS:
		return None, leafType(dataSet)
	#检查两个切分后的子集，如果大小小于用户定义的参数tolN，则不切分
	mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
	if(shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
		return None, leafType(dataSet)
	return bestIndex, bestValue#返回切分特征和特征值
#将数据集格式化为目标变量Y和自变量X
def linearSolve(dataSet):
	m,n = shape(dataSet)
	X = mat(ones((m,n)))
	Y = mat(ones((m,1)))
	X[:,1:n] = dataSet[:,0:n-1]
	Y = dataSet[:,-1]
	xTx = X.T * X
	if linalg.det(xTx) == 0.0:
		raise NameError('This matrix is singular, cannot do inverse,\n\
					try increasing the second value of ops')
	ws = xTx.I * (X.T * Y)
	return ws, X, Y

def modelLeaf(dataSet):
	ws, X, Y = linearSolve(dataSet)
	return ws

def modelError(dataSet):
	ws, X, Y = linearSolve(dataSet)
	yHat = X * ws
	return sum(power(Y - yHat, 2))

def binSplitDataSet(dataSet, feature, value):#数据集合，待切分特征，特征值
	mat0 = dataSet[nonzero(dataSet[:,feature] > value)[0],:]
	mat1 = dataSet[nonzero(dataSet[:,feature] <= value)[0],:]
	return mat0, mat1

def createTree(dataSet, leafType = regLeaf, errType = regErr, ops = (1,4)):
	feat, val = chooseBestSplit(dataSet, leafType, errType, ops)
	if feat == None:
		return val
	retTree = {}
	retTree['spInd'] = feat
	retTree['spVal'] = val
	lSet, rSet = binSplitDataSet(dataSet, feat, val)
	retTree['left'] = createTree(lSet, leafType, errType, ops)
	retTree['right'] = createTree(rSet, leafType, errType, ops)
	return retTree


In [2]:
testMat = mat(eye(4))
testMat

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [3]:
testMat[nonzero(testMat[:,1] < 0.5)[0],:]

matrix([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [4]:
mat0, mat1 = binSplitDataSet(testMat,1,0.5)
mat0

matrix([[0., 1., 0., 0.]])

In [5]:
mat1

matrix([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [6]:
myDat = loadDataSet('E:\python\machinelearning\MLDownloads\machinelearninginaction\Ch09\ex00.txt')
myDat = mat(myDat)

In [7]:
myDat

matrix([[ 3.609800e-02,  1.550960e-01],
        [ 9.933490e-01,  1.077553e+00],
        [ 5.308970e-01,  8.934620e-01],
        [ 7.123860e-01,  5.648580e-01],
        [ 3.435540e-01, -3.717000e-01],
        [ 9.801600e-02, -3.327600e-01],
        [ 6.911150e-01,  8.343910e-01],
        [ 9.135800e-02,  9.993500e-02],
        [ 7.270980e-01,  1.000567e+00],
        [ 9.519490e-01,  9.452550e-01],
        [ 7.685960e-01,  7.602190e-01],
        [ 5.413140e-01,  8.937480e-01],
        [ 1.463660e-01,  3.428300e-02],
        [ 6.731950e-01,  9.150770e-01],
        [ 1.835100e-01,  1.848430e-01],
        [ 3.395630e-01,  2.067830e-01],
        [ 5.179210e-01,  1.493586e+00],
        [ 7.037550e-01,  1.101678e+00],
        [ 8.307000e-03,  6.997600e-02],
        [ 2.439090e-01, -2.946700e-02],
        [ 3.069640e-01, -1.773210e-01],
        [ 3.649200e-02,  4.081550e-01],
        [ 2.955110e-01,  2.882000e-03],
        [ 8.375220e-01,  1.229373e+00],
        [ 2.020540e-01, -8.774400e-02],


In [8]:
import matplotlib.pyplot as plt 
fig = plt.figure()
ax = fig.add_subplot(111)
#ax.scatter(xMat[:,1].flatten().A[0],mat(yArr).T.flatten().A[0], s = 2, c = 'green')
#ax.plot(xSort[:,1], yHat[srtInd])
#plt.show()

In [9]:
createTree(myDat,ops=(0,1))

{'spInd': 0,
 'spVal': 0.48813,
 'left': {'spInd': 0,
  'spVal': 0.620599,
  'left': {'spInd': 0,
   'spVal': 0.625336,
   'left': {'spInd': 0,
    'spVal': 0.625791,
    'left': {'spInd': 0,
     'spVal': 0.643601,
     'left': {'spInd': 0,
      'spVal': 0.651376,
      'left': {'spInd': 0,
       'spVal': 0.6632,
       'left': {'spInd': 0,
        'spVal': 0.683921,
        'left': {'spInd': 0,
         'spVal': 0.819823,
         'left': {'spInd': 0,
          'spVal': 0.837522,
          'left': {'spInd': 0,
           'spVal': 0.846455,
           'left': {'spInd': 0,
            'spVal': 0.919384,
            'left': {'spInd': 0,
             'spVal': 0.976414,
             'left': {'spInd': 0,
              'spVal': 0.985425,
              'left': {'spInd': 0,
               'spVal': 0.989888,
               'left': {'spInd': 0,
                'spVal': 0.993349,
                'left': 1.035533,
                'right': 1.077553},
               'right': {'spInd': 0,
        

In [10]:
myDat1 = loadDataSet('E:\python\machinelearning\MLDownloads\machinelearninginaction\Ch09\ex0.txt')
myDat1 = mat(myDat1)

In [11]:
createTree(myDat1)

{'spInd': 1,
 'spVal': 0.39435,
 'left': {'spInd': 1,
  'spVal': 0.582002,
  'left': {'spInd': 1,
   'spVal': 0.797583,
   'left': 3.9871632,
   'right': 2.9836209534883724},
  'right': 1.980035071428571},
 'right': {'spInd': 1,
  'spVal': 0.197834,
  'left': 1.0289583666666666,
  'right': -0.023838155555555553}}

In [12]:
createTree(myDat,ops=(0.5,12))

{'spInd': 0,
 'spVal': 0.48813,
 'left': 1.0180967672413792,
 'right': -0.04465028571428572}

In [13]:
myDat2 = loadDataSet('E:\python\machinelearning\MLDownloads\machinelearninginaction\Ch09\ex2.txt')
myDat2 = mat(myDat2)
createTree(myDat2)

{'spInd': 0,
 'spVal': 0.499171,
 'left': {'spInd': 0,
  'spVal': 0.729397,
  'left': {'spInd': 0,
   'spVal': 0.952833,
   'left': {'spInd': 0,
    'spVal': 0.958512,
    'left': 105.24862350000001,
    'right': 112.42895575000001},
   'right': {'spInd': 0,
    'spVal': 0.759504,
    'left': {'spInd': 0,
     'spVal': 0.790312,
     'left': {'spInd': 0,
      'spVal': 0.833026,
      'left': {'spInd': 0,
       'spVal': 0.944221,
       'left': 87.3103875,
       'right': {'spInd': 0,
        'spVal': 0.85497,
        'left': {'spInd': 0,
         'spVal': 0.910975,
         'left': 96.452867,
         'right': {'spInd': 0,
          'spVal': 0.892999,
          'left': 104.825409,
          'right': {'spInd': 0,
           'spVal': 0.872883,
           'left': 95.181793,
           'right': 102.25234449999999}}},
        'right': 95.27584316666666}},
      'right': {'spInd': 0,
       'spVal': 0.811602,
       'left': 81.110152,
       'right': 88.78449880000001}},
     'right': 102.

In [14]:
createTree(myDat2,ops=(10000,4))

{'spInd': 0,
 'spVal': 0.499171,
 'left': 101.35815937735848,
 'right': -2.637719329787234}

In [15]:
def isTree(obj):#测试输入变量是否是一棵树：判断当前处理的节点是否为叶节点
	return (type(obj).__name__=='dict')

def getMean(tree):#遍历直到叶节点。对树进行塌陷处理（返回树平均值）
	if isTree(tree['right']): tree['right'] = getMean(tree['right'])
	if isTree(tree['left']): tree['left'] = getMean(tree['left'])
	return (tree['left'] + tree['right']) / 2.0

def prune(tree, testData):
	if shape(testData)[0] == 0: 
		print("testData:",testData)
		return getMean(tree)#测试数据集是否为空
	#检查某个分支到底是子树还是节点
	if (isTree(tree['right']) or isTree(tree['left'])):
		lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
		#print("lSet: ",lSet)
		#print("rSet: ",rSet)
	#如果是子树，就调用函数prune()来对该子树进行剪枝
	if isTree(tree['left']):
		tree['left'] = prune(tree['left'], lSet)
	if isTree(tree['right']):
		tree['right'] = prune(tree['right'],rSet)
	#如果两个分支已经不再是子树，那么就可以进行合并。
	#具体做法是对合并前后的误差进行比较。如果合并后的误差比不合并的误差小就进行合并操作，反之则不合并直接返回。
	if not isTree(tree['left']) and not isTree(tree['right']):
		lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
		#对合并前后的误差进行比较：合并后的误差比不合并的误差小就进行合并操作，反之则不合并直接返回
		errorNoMerge = sum(power(lSet[:,-1] - tree['left'], 2)) + sum(power(rSet[:,-1] - tree['right'], 2))
		treeMean = (tree['left'] + tree['right']) / 2.0
		errorMerge = sum(power(testData[:,-1] - treeMean, 2))
		if errorMerge < errorNoMerge:
			print("Merging")
			return treeMean
		else:
			print("returnla")
			return tree
	else:
		return tree

In [16]:
myTree = createTree(myDat2,ops = (0,1))

In [17]:
myDatTest = loadDataSet('E:\python\machinelearning\MLDownloads\machinelearninginaction\Ch09\ex2test.txt')
myMat2Test = mat(myDatTest)
myMat2Test

matrix([[ 4.21862000e-01,  1.08302410e+01],
        [ 1.05349000e-01, -2.24161100e+00],
        [ 1.55196000e-01,  2.18729760e+01],
        [ 1.61152000e-01,  2.01541800e+00],
        [ 3.82632000e-01, -3.87789790e+01],
        [ 1.77100000e-02,  2.01091130e+01],
        [ 1.29656000e-01,  1.52668870e+01],
        [ 6.13926000e-01,  1.11900063e+02],
        [ 4.09277000e-01,  1.87473100e+00],
        [ 8.07556000e-01,  1.11223754e+02],
        [ 5.93722000e-01,  1.33835486e+02],
        [ 9.53239000e-01,  1.10465070e+02],
        [ 2.57402000e-01,  1.53328990e+01],
        [ 6.45385000e-01,  9.39830540e+01],
        [ 5.63460000e-01,  9.36452770e+01],
        [ 4.08338000e-01, -3.07198780e+01],
        [ 8.74394000e-01,  9.18735050e+01],
        [ 2.63805000e-01, -1.92752000e-01],
        [ 4.11198000e-01,  1.07511180e+01],
        [ 4.49884000e-01,  9.21190100e+00],
        [ 6.46315000e-01,  1.13533660e+02],
        [ 6.73718000e-01,  1.25135638e+02],
        [ 8.05148000e-01,  1.133

In [18]:
prune(myTree,myMat2Test)

Merging
returnla
testData: []
Merging
Merging
Merging
returnla
Merging
Merging
returnla
returnla
Merging
returnla
Merging
Merging
Merging
Merging
Merging
returnla
returnla
Merging
returnla
returnla
Merging
Merging
testData: []
Merging
Merging
returnla
Merging
testData: []
Merging
returnla
returnla
Merging
Merging
testData: []
returnla
returnla
returnla
returnla
Merging
Merging
returnla
Merging
testData: []
returnla
returnla
returnla
Merging
returnla
returnla
returnla
testData: []
Merging
Merging
Merging
returnla
Merging
returnla
Merging
returnla
Merging
testData: []
returnla
testData: []
Merging
returnla
returnla
returnla
Merging
returnla
Merging
Merging
Merging
Merging
testData: []
Merging
returnla
Merging
returnla
returnla
Merging
returnla
Merging
returnla
Merging
Merging
Merging


{'spInd': 0,
 'spVal': 0.499171,
 'left': {'spInd': 0,
  'spVal': 0.729397,
  'left': {'spInd': 0,
   'spVal': 0.952833,
   'left': {'spInd': 0,
    'spVal': 0.965969,
    'left': 92.5239915,
    'right': {'spInd': 0,
     'spVal': 0.956951,
     'left': {'spInd': 0,
      'spVal': 0.958512,
      'left': {'spInd': 0,
       'spVal': 0.960398,
       'left': 112.386764,
       'right': 123.559747},
      'right': 135.837013},
     'right': 111.2013225}},
   'right': {'spInd': 0,
    'spVal': 0.759504,
    'left': {'spInd': 0,
     'spVal': 0.763328,
     'left': {'spInd': 0,
      'spVal': 0.769043,
      'left': {'spInd': 0,
       'spVal': 0.790312,
       'left': {'spInd': 0,
        'spVal': 0.806158,
        'left': {'spInd': 0,
         'spVal': 0.815215,
         'left': {'spInd': 0,
          'spVal': 0.833026,
          'left': {'spInd': 0,
           'spVal': 0.841547,
           'left': {'spInd': 0,
            'spVal': 0.841625,
            'left': {'spInd': 0,
            

In [19]:
myDat2 = loadDataSet('E:\python\machinelearning\MLDownloads\machinelearninginaction\Ch09\exp2.txt')
myMat2 = mat(myDat2)
myMat2

matrix([[7.0670000e-02, 3.4708290e+00],
        [5.3407600e-01, 6.3771320e+00],
        [7.4722100e-01, 8.9494070e+00],
        [6.6897000e-01, 8.0340810e+00],
        [5.8608200e-01, 6.9977210e+00],
        [7.6496200e-01, 9.3181100e+00],
        [6.5812500e-01, 7.8803330e+00],
        [3.4673400e-01, 4.2133590e+00],
        [3.1396700e-01, 3.7624960e+00],
        [6.0141800e-01, 7.1888050e+00],
        [4.0439600e-01, 4.8934030e+00],
        [1.5434500e-01, 3.6831750e+00],
        [9.8406100e-01, 1.1712928e+01],
        [5.9751400e-01, 7.1466940e+00],
        [5.1440000e-03, 3.3331500e+00],
        [1.4229500e-01, 3.7436810e+00],
        [2.8000700e-01, 3.7373760e+00],
        [5.4200800e-01, 6.4942750e+00],
        [4.6678100e-01, 5.5322550e+00],
        [7.0697000e-01, 8.4767180e+00],
        [1.9103800e-01, 3.6739210e+00],
        [7.5659100e-01, 9.1767220e+00],
        [9.1287900e-01, 1.0850358e+01],
        [5.2470100e-01, 6.0674440e+00],
        [3.0609000e-01, 3.6811480e+00],


In [20]:
createTree(myMat2,modelLeaf,modelError,(1,10))

{'spInd': 0, 'spVal': 0.285477, 'left': matrix([[1.69855694e-03],
         [1.19647739e+01]]), 'right': matrix([[3.46877936],
         [1.18521743]])}

In [21]:
def regTreeEval(model, inDat):
	return float(model)

def modelTreeEval(model, inDat):
	n = shape(inDat)[1]
	X = mat(ones((1, n+1)))
	X[:,1:n+1] = inDat
	return float(X * model)

def treeForeCast(tree, inData, modelEval = regTreeEval):
	if not isTree(tree):
		return modelEval(tree, inData)
	if inData[tree['spInd']] > tree['spVal']:
		if isTree(tree['left']):
			return treeForeCast(tree['left'], inData, modelEval)
		else:
			return modelEval(tree['left'], inData)
	else:
		if isTree(tree['right']):
			return treeForeCast(tree['right'], inData, modelEval)
		else:
			return modelEval(tree['right'], inData)

def createForeCast(tree, testData, modelEval = regTreeEval):
	m = len(testData)
	yHat = mat(zeros((m,1)))
	for i in range(m):
		yHat[i,0] = treeForeCast(tree, mat(testData[i]), modelEval)
	return yHat

In [22]:
trainMat = mat(loadDataSet('E:\\python\\machinelearning\\MLDownloads\\machinelearninginaction\\Ch09\\bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('E:\\python\\machinelearning\\MLDownloads\\machinelearninginaction\\Ch09\\bikeSpeedVsIq_test.txt'))
myTree = createTree(trainMat, ops=(1,20))
yHat = createForeCast(myTree, testMat[:,0])
print("系数1：",corrcoef(yHat,testMat[:,1],rowvar=0)[0,1])

系数1： 0.9640852318222141


In [23]:
myTree = createTree(trainMat, modelLeaf, modelError, (1,20))
yHat = createForeCast(myTree, testMat[:,0], modelTreeEval)
print("系数2：", corrcoef(yHat, testMat[:,1],rowvar=0)[0,1])

系数2： 0.9760412191380593


In [24]:
ws, X, Y =linearSolve(trainMat)
print(ws)
for i in range(shape(testMat)[0]):
	yHat[i] = testMat[i,0] * ws[1,0] + ws[0,0]
print("系数3：",corrcoef(yHat,testMat[:,1],rowvar=0)[0,1])

[[37.58916794]
 [ 6.18978355]]
系数3： 0.9434684235674763


In [ ]:
from tkinter import *
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

def reDraw(tolS, tolN):
	reDraw.f.clf()#清空绘图区
	reDraw.a = reDraw.f.add_subplot(111)
	if chkBtnVar.get():
		if tolN < 2:
			tolN = 2
		myTree = createTree(reDraw.rawDat, modelLeaf,modelError,(tolS,tolN))
		yHat = createForeCast(myTree,reDraw.testDat,modelTreeEval)
	else:
		myTree = createTree(reDraw.rawDat, ops=(tolS,tolN))
		yHat = createForeCast(myTree, reDraw.testDat)
	reDraw.a.scatter(reDraw.rawDat[:,0].flatten().A[0], reDraw.rawDat[:,1].flatten().A[0],s = 5)
	reDraw.a.plot(reDraw.testDat, yHat, linewidth = 2.0)
	reDraw.canvas.show()

def getInputs():
	try: tolN = int(tolNentry.get())
	except:
		tolN = 10
		print("enter integer for tolN")
		tolNentry.delete(0,END)
		tolNentry.insert(0,'10')
	try:
		tolS = float(tolSentry.get())
	except:
		tolS = 1.0
		print("enter float for tolS")
		tolSentry.delete(0,END)
		tolSentry.insert(0,'1.0')
	return tolN, tolS

def drawNewTree():
	tolN, tolS = getInputs()
	reDraw(tolS,tolN)

root = Tk()

reDraw.f = Figure(figsize = (5,4), dpi = 100)
reDraw.canvas = FigureCanvasTkAgg(reDraw.f, master = root)
reDraw.canvas.show()
reDraw.canvas.get_tk_widget().grid(row = 0, columnspan=3)
#Label(root, text = "Plot Place Holder").grid(row = 0, columnspan = 3)

Label(root, text = "tolN:").grid(row = 1, column = 0)
tolNentry = Entry(root)
tolNentry.grid(row = 1, column = 1)
tolNentry.insert(0,'10')

Label(root, text = "tolS:").grid(row = 2, column = 0)
tolSentry = Entry(root)
tolSentry.grid(row = 2, column = 1)
tolSentry.insert(0,'1.0')

Button(root, text = "ReDraw", command = drawNewTree).grid(row = 1, column = 2, rowspan = 3)
chkBtnVar = IntVar()
chkBtn = Checkbutton(root, text = "Model Tree", variable = chkBtnVar)
chkBtn.grid(row = 3, column = 0, columnspan = 2)

Button(root, text = 'Quit', fg = "blue", command = root.quit).grid(row = 0, column = 2)

reDraw.rawDat = mat(loadDataSet('E:\\python\\machinelearning\\MLDownloads\\machinelearninginaction\\Ch09\\sine.txt'))
reDraw.testDat = arange(min(reDraw.rawDat[:,0]), max(reDraw.rawDat[:,0]),0.01)
reDraw(1.0,10)

root.mainloop()